
# Deploy Dolly Model with high performance on SageMaker 

In this notebook, we explore how to host a large language model on SageMaker using the latest container that packages some of the most popular open source libraries for model parallel inference like DeepSpeed and Hugging Face Accelerate. We use DJLServing as the model serving solution in this example. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).

Language models have recently exploded in both size and popularity. In 2018, BERT-large entered the scene and, with its 340M parameters and novel transformer architecture, set the standard on NLP task accuracy. Within just a few years, state-of-the-art NLP model size has grown by more than 500x with models such as OpenAI’s 175 billion parameter GPT-3 and similarly sized open source Bloom 176B raising the bar on NLP accuracy. This increase in the number of parameters is driven by the simple and empirically-demonstrated positive relationship between model size and accuracy: more is better. With easy access from models zoos such as Hugging Face and improved accuracy in NLP tasks such as classification and text generation, practitioners are increasingly reaching for these large models. However, deploying them can be a challenge because of their size.

Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. This notebook uses tensor parallelism techniques which allow GPUs to work simultaneously on the same layer of a model and achieve low latency inference relative to a pipeline parallel solution.

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy the open source [Dolly-v2-12b](https://huggingface.co/databricks/dolly-v2-12b) model across GPUs on a ml.g5.24xlarge instance. 

Databricks dolly-v2-12b is an instruction-following large language model trained on the Databricks machine learning platform. Based on pythia-12b, Dolly is trained on ~15k instruction/response fine tuning records databricks-dolly-15k generated by Databricks employees in capability domains from the InstructGPT paper, including brainstorming, classification, closed QA, generation, information extraction, open QA and summarization. dolly-v2-12b is not a state-of-the-art model, but does exhibit surprisingly high quality instruction following behavior not characteristic of the foundation model on which it is based. You can read more information about Dolly [here](https://github.com/databrickslabs/dolly#getting-started-with-response-generation)

DeepSpeed is used for tensor parallelism inference while DJLServing handles inference requests and the distributed workers. For further reading on DeepSpeed you can refer to https://arxiv.org/pdf/2207.00032.pdf


## Licence agreement
 - View license information https://github.com/databrickslabs/dolly/blob/master/LICENSE before using the model.
 - This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0. 

 
 


In [2]:
!pip install sagemaker boto3 huggingface_hub --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
awscli 1.27.111 requires botocore==1.29.111, but you have botocore 1.29.121 which is incompatible.
awscli 1.27.111 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.121 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

In [4]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "hf-large-model-djl-/code_dolly-v2-12b"  # folder within bucket where code artifact will go

s3_model_prefix = "hf-large-model-djl-/model_dolly-v2-12b"  # folder within bucket where code artifact will go
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

Pretrained model will be uploaded to ---- > s3://sagemaker-us-east-1-948014026119/hf-large-model-djl-/model_dolly-v2-12b/


### Download the model from Hugging Face and upload the model artifacts on Amazon S3

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = "databricks/dolly-v2-12b"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

In [7]:
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

Model uploaded to --- > s3://sagemaker-us-east-1-948014026119/hf-large-model-djl-/model_dolly-v2-12b
We will set option.s3url=s3://sagemaker-us-east-1-948014026119/hf-large-model-djl-/model_dolly-v2-12b


In [8]:
#!rm -rf {model_download_path}

## Create SageMaker compatible Model artifact,  upload Model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

However in this notebook, we demonstrate how to deploy a model with custom inference code.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties` and `model.py`.

The tarball is in the following format

```
code
├──── 
│   └── serving.properties
│   └── model.py
   

```

- `serving.properties` is the configuration file that can be used to configure the model server.
- `model.py` is the script handles any requests for serving.


#### Create serving.properties 

This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -
- `engine`: The engine for DJL to use. In this case, it is **DeepSpeed**.
- `option.entryPoint`: The entrypoint python file or module. This should align with the engine that is being used. 
- `option.s3url`: Set this to the URI of the Amazon S3 bucket that contains the model. 

If you want to download the model from huggingface.co, you can set `option.modelid`. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. 
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which DeepSpeed needs to partition the model. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests. For further reading on DeepSpeedyou can follow the link https://www.deepspeed.ai/tutorials/inference-tutorial/#initializing-for-inference. 

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.


DeepSpeed uses TensorParallelism where individual layers (Tensors) are sharded accross devices. For example each GPU can have a slice of each layer. 

The data is sent to all GPU devices where a partial result is computed on each GPU. The partial results are then collected though an All-Gather operation to compute the final result. 
TensorParallelism generally provides higher GPU utilization and better performance.


In [9]:
%%writefile serving.properties
engine = DeepSpeed
option.tensor_parallel_degree = 4
option.s3url = {{s3url}}

Overwriting serving.properties


In [10]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("serving.properties").open().read())
Path("serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize serving.properties | cat -n

     1	engine = DeepSpeed
     2	option.tensor_parallel_degree = 4
     3	option.s3url = s3://sagemaker-us-east-1-948014026119/hf-large-model-djl-/model_dolly-v2-12b/


In [11]:
%%writefile model.py
from djl_python import Input, Output
import deepspeed
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from instruct_pipeline import InstructionTextGenerationPipeline

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    logging.info(f"Loading model in {model_location}")

    tokenizer = AutoTokenizer.from_pretrained(model_location)
    dtype = torch.float16

    model = AutoModelForCausalLM.from_pretrained(
        model_location, low_cpu_mem_usage=True, torch_dtype=dtype
    )

    if "dtype" in properties:
        if properties["dtype"] == "float16":
            model.to(torch.float16)
        if properties["dtype"] == "bfloat16":
            model.to(torch.bfloat16)

    logging.info(f"Starting DeepSpeed init with TP={tensor_parallel}")
    model = deepspeed.init_inference(
        model,
        mp_size=tensor_parallel,
        dtype=model.dtype,
        replace_method="auto",
        replace_with_kernel_inject=True,
    )
    return model.module, tokenizer

model = None
tokenizer = None
generator = None


def run_inference(model, tokenizer, data, params):
    generate_kwargs = params
    tokenizer.pad_token = tokenizer.eos_token
    input_tokens = tokenizer.batch_encode_plus(data, return_tensors="pt", padding=True)
    for t in input_tokens:
        if torch.is_tensor(input_tokens[t]):
            input_tokens[t] = input_tokens[t].to(torch.cuda.current_device())
    outputs = model.generate(**input_tokens, **generate_kwargs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None
    data = inputs.get_as_json()

    input_sentences = data["inputs"]
    params = data["parameters"]

    outputs = run_inference(model, tokenizer, input_sentences, params)
    result = {"outputs": outputs}
    return Output().add_as_json(result)

Overwriting model.py


**Image URI for the DJL container is being used here**

In [12]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=sess.boto_session.region_name, version="0.21.0"
)

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117


**Create the Tarball and then upload to S3 location**

In [13]:
%%sh
mkdir code_dolly-12b_deepspeed
cp serving.properties code_dolly-12b_deepspeed/
# remove the following lines if not needed
cp model.py code_dolly-12b_deepspeed/
cp instruct_pipeline.py code_dolly-12b_deepspeed/
tar czvf model.tar.gz code_dolly-12b_deepspeed/
rm -rf code_dolly-12b_deepspeed

code_dolly-12b_deepspeed/
code_dolly-12b_deepspeed/instruct_pipeline.py
code_dolly-12b_deepspeed/model.py
code_dolly-12b_deepspeed/serving.properties


mkdir: cannot create directory ‘code_dolly-12b_deepspeed’: File exists


In [14]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-948014026119/hf-large-model-djl-/code_dolly-v2-12b/model.tar.gz


### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g5.24xlarge 
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 2400 to ensure health check starts after the model is ready    
3. Create the end point using the endpoint config created    
    

#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the instance because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB. It leverages `s5cmd`(https://github.com/peak/s5cmd) which offers a very fast download speed and hence extremely useful when downloading large models.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.


In [15]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"dolly-v2-12b")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

dolly-v2-12b-2023-04-27-12-04-09-491
Created Model: arn:aws:sagemaker:us-east-1:948014026119:model/dolly-v2-12b-2023-04-27-12-04-09-491


In [16]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.24xlarge",
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 300,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:948014026119:endpoint-config/dolly-v2-12b-2023-04-27-12-04-09-491-config',
 'ResponseMetadata': {'RequestId': 'fb906cf9-e1b1-4fef-93a2-75cacbc5cbf5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb906cf9-e1b1-4fef-93a2-75cacbc5cbf5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Thu, 27 Apr 2023 12:04:10 GMT'},
  'RetryAttempts': 0}}

In [17]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:948014026119:endpoint/dolly-v2-12b-2023-04-27-12-04-09-491-endpoint


### This step can take ~ 10 min or longer so please be patient

In [18]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:948014026119:endpoint/dolly-v2-12b-2023-04-27-12-04-09-491-endpoint
Status: InService


#### While you wait for the endpoint to be created, you can read more about:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [DeepSpeed](https://www.deepspeed.ai/tutorials/inference-tutorial/#initializing-for-inference)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results.

You can pass a batch of prompts as input to the model. This done by setting `inputs` to the list of prompts. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters. These `parameters` need to be passed to the endpoint as a dictionary of `kwargs`. Refer this documentation - https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig for more details.

The below code sample illustrates the invocation of the endpoint using a prompts and also sets some parameters.


In [19]:
%%time
prompts = [
    "Explain to me the difference between nuclear fission and fusion"
]
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": prompts,
            "parameters": {
                "max_length": 150,
            },
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 3.2 s


'{\n  "outputs":[\n    "Explain to me the difference between nuclear fission and fusion reaction\\n\\n nuclear fission chain reaction.\\n\\nNuclear fission is a nuclear chain reaction in which a large atomic nucleus splits into two or more smaller nuclei. This is called fission and it is called fission reaction. Fission. Fission. In this process a neutron is also produced. In this process a lot of energy is also released. In fission reaction. Fission is also called nuclear fission. Fission. In this process also called fission. In this process  nuclear fission.\\nNuclear fusion is also a nuclear fusion reaction in which two or more nuclei join together to form one bigger nucleus. This join called fusion. In this fusion reaction. In this process a  two or two"\n  ]\n}'

## Clean Up

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)